<a href="https://colab.research.google.com/github/Mun-Robbery/AAT/blob/main/Utilizing_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gc; gc.enable()
from IPython.display import clear_output
import pandas as pd
train = pd.read_csv('train.csv')

In [4]:
train.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


Pooler Output

In [5]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel,
    AutoConfig,
    AutoTokenizer,
    logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
pooler_output = outputs[1]
logits = nn.Linear(config.hidden_size, 1)(pooler_output) # regression head

print(f'Pooler Output Shape: {pooler_output.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features, outputs
gc.collect();

Pooler Output Shape: (16, 768)
Logits Shape: (16, 1)


Last Hidden State Output

In [6]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel, AutoConfig,
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
last_hidden_state = outputs[0]
cls_embeddings = last_hidden_state[:, 0]
logits = nn.Linear(config.hidden_size, 1)(cls_embeddings) # regression head

print(f'Last Hidden State Output Shape: {last_hidden_state.detach().numpy().shape}')
print(f'CLS Embeddings Output Shape: {cls_embeddings.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features, outputs
gc.collect();

Last Hidden State Output Shape: (16, 256, 768)
CLS Embeddings Output Shape: (16, 768)
Logits Shape: (16, 1)


Mean Pooling

In [7]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel, AutoConfig,
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
last_hidden_state = outputs[0]
attention_mask = features['attention_mask']